In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def extract_video_first_frame(path, use_cv2=True):
    
    cap = cv2.VideoCapture(path)
    
    ret, frame = cap.read()
    
    if use_cv2:
        cv2.imshow('First Frame', frame)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(new_frame)
        plt.axis('off')
        
    return frame

In [3]:
# PATH = './data/videos/rcpc_bimbambum.mp4'
# img = extract_video_first_frame(PATH)

In [14]:
proto_file = './openpose/openpose-master/models/pose/coco/pose_deploy_linevec.prototxt'
weight_file = './openpose/openpose-master/models/pose/coco/pose_iter_440000.caffemodel'

img = cv2.imread('./data/basketball_2.jpg')
img_copy = img.copy()

In [15]:
frame_width, frame_height = img.shape[1], img.shape[0]
in_height = 368
in_width = 368

pose_net = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
pose_blob = cv2.dnn.blobFromImage(img_copy, scalefactor=1.0/255, size=(in_width, in_height), mean=(0,0,0), swapRB=False, crop=False)
pose_net.setInput(pose_blob)
output = pose_net.forward()

In [16]:
len(output)

1

# Single Point

In [ ]:
i = 0

prob_map = output[0,i,:,:]
prob_map = cv2.resize(prob_map, (frame_width, frame_height))

plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))    
plt.imshow(prob_map, alpha=0.5)
plt.show()

In [ ]:
threshold = 0.1

map_gauss = cv2.GaussianBlur(prob_map, (3,3), 0, 0)
map_mask = np.uint8(map_gauss > threshold)

In [ ]:
contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
body_points = []

for c in contours:
    blob_mask = np.zeros(map_mask.shape)
    blob_mask = cv2.fillConvexPoly(blob_mask, c, 1)
    masked_prob_map = map_gauss * blob_mask
    _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)
    
    body_points.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))

In [ ]:
for i in range(len(body_points)):
    cv2.circle(img_copy, (body_points[i][0], body_points[i][1]), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)

cv2.imshow('Body Points', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Multiple Points

In [17]:
img_copy = img.copy()
n_points = 18
key_points_mapping = ['Nose', 'Neck', 'R-Shoulder', 'R-Elbow', 'R-Wrist', 'L-Shoulder', 'L-Elbow', 'L-Wrist', 'R-Hip', 'R-Knee', 'R-Ankle',
                     'L-Hip', 'L-Knee', 'L-Ankle', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']


POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16]]

# index of pafs correspoding to the POSE_PAIRS
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44],
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30],
          [47,48], [49,50], [53,54], [51,52], [55,56],
          [37,38], [45,46]]
 
colors = [[0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

In [18]:
def get_key_points(prob_map, threshold=0.1):
    body_points = []
    
    map_gauss = cv2.GaussianBlur(prob_map, (3,3), 0, 0)
    map_mask = np.uint8(map_gauss > threshold)
    
    contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        blob_mask = np.zeros(map_mask.shape)
        blob_mask = cv2.fillConvexPoly(blob_mask, c, 1)
        masked_prob_map = map_gauss * blob_mask
        _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)

        body_points.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))

    return body_points

In [19]:
def get_valid_pairs(output):
    
    n_inter_samples = 10
    paf_threshold = 0.1
    conf_threshold = 0.7
    
    valid_pairs = []
    invalid_pairs = []
    
    for k in range(len(mapIdx)):
        paf_A = output[0, mapIdx[k][0], :, :]
        paf_B = output[0, mapIdx[k][1], :, :]
        paf_A = cv2.resize(paf_A, (frame_width, frame_height))
        paf_B = cv2.resize(paf_B, (frame_width, frame_height))
        
        candidate_A = detected_key_points[POSE_PAIRS[k][0]]
        candidate_B = detected_key_points[POSE_PAIRS[k][1]]
        len_candidate_A = len(candidate_A)
        len_candidate_B = len(candidate_B)
        
        if len_candidate_A != 0 and len_candidate_B != 0:
            valid_pair = np.zeros((0,3))
            
            for i in range(len_candidate_A):
                max_j = -1
                max_score = -1
                found = False
                
                for j in range(len_candidate_B):
                    d_ij = np.subtract(candidate_B[j][:2], candidate_A[i][:2])
                    norm = np.linalg.norm(d_ij)
                    if norm:
                        d_ij = d_ij / norm
                    else:
                        continue
                        
                    inter_coord = list(zip(np.linspace(candidate_A[i][0], candidate_B[j][0], num=n_inter_samples),
                                           np.linspace(candidate_A[i][1], candidate_B[j][1], num=n_inter_samples)))
                    
                    paf_inter = []
                    for k in range(len(inter_coord)):
                        paf_inter.append([paf_A[int(round(inter_coord[k][1])), int(round(inter_coord[k][0]))],
                                         paf_B[int(round(inter_coord[k][1])), int(round(inter_coord[k][0]))]])
                        
                    paf_scores = np.dot(paf_inter, d_ij)
                    avg_paf_score = sum(paf_scores) / len(paf_scores)
                    
                    if (len(np.where(paf_scores > paf_threshold)[0]) / n_inter_samples) > conf_threshold:
                         if avg_paf_score > max_score:
                                max_j = j
                                max_score = avg_paf_score
                                found = True
                                
                if found:
                    valid_pair = np.append(valid_pair, [[candidate_A[i][3], candidate_B[max_j][3], max_score]], axis=0)
                    
            valid_pairs.append(valid_pair)
        
        else:
            print('No connection found: k = {}'.format(k))
            invalid_pairs.append(k)
            valid_pairs.append([])
            
    return valid_pairs, invalid_pairs 

In [20]:
def assign_keypoints(valid_pairs, invalid_pairs):
    person_wise_keypoints = -1 * np.ones((0,19))
    
    for k in range(len(mapIdx)):
        if k not in invalid_pairs:
            part_A = valid_pairs[k][:,0]
            part_B = valid_pairs[k][:,1]
            idx_A, idx_B = np.array(POSE_PAIRS[k])
            
            for i in range(len(valid_pairs[k])):
                found = False
                person_idx = -1
                for j in range(len(person_wise_keypoints)):
                    if person_wise_keypoints[j][idx_A] == part_A[i]:
                        person_idx = j
                        found = True
                        break
                
                if found:
                    person_wise_keypoints[person_idx][idx_B] == part_B[i]
                    person_wise_keypoints[person_idx][-1] += key_points_list[part_B[i].astype(int), 2] + valid_pairs[k][i][2]
                    
                elif not found and k < 17:
                    row = -1 * np.ones(19)
                    row[idx_A] = part_A[i]
                    row[idx_B] = part_B[i]
                    
                    row[-1] = sum(key_points_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                    person_wise_keypoints = np.vstack([person_wise_keypoints, row])
                    
    return person_wise_keypoints

In [21]:
frame_width = img_copy.shape[1]
frame_height = img_copy.shape[0]
in_height = 368
in_width = int((in_height/frame_height) * frame_width)

pose_model = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
input_blob = cv2.dnn.blobFromImage(img_copy, 1.0/255, (in_width, in_height), (0,0,0), swapRB=False, crop=False)

pose_model.setInput(input_blob)
output = pose_model.forward()

detected_key_points = []
key_points_list = np.zeros((0,3))
key_point_id = 0

for i in range(n_points):
    prob_map = output[0,i,:,:]
    prob_map = cv2.resize(prob_map, (img_copy.shape[1], img_copy.shape[0]))
    body_points = get_key_points(prob_map)
    print('Key points - {} : {}'.format(key_points_mapping[i], body_points))
    
    key_points_with_id = []
    for j in range(len(body_points)):
        key_points_with_id.append(body_points[j] + (key_point_id,))
        key_points_list = np.vstack([key_points_list, body_points[j]])
        
        key_point_id += 1
        
    detected_key_points.append(key_points_with_id)
    
for i in range(n_points):
    for j in range(len(detected_key_points[i])):
        cv2.circle(img_copy, detected_key_points[i][j][0:2], 5, colors[i], -1, cv2.LINE_AA)
        
cv2.imshow('Key Points', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

valid_pairs, invalid_pairs = get_valid_pairs(output)
print('\nValid Pairs:', valid_pairs)
print('Invalid Pairs:', invalid_pairs)

Key points - Nose : [(970, 350, 0.8470357)]
Key points - Neck : [(971, 515, 0.7220071)]
Key points - R-Shoulder : [(805, 516, 0.75325483)]
Key points - R-Elbow : [(846, 888, 0.5750528)]
Key points - R-Wrist : [(928, 1054, 0.10203365)]
Key points - L-Shoulder : [(1176, 475, 0.6710307)]
Key points - L-Elbow : [(1259, 724, 0.68433505)]
Key points - L-Wrist : [(1423, 971, 0.63367397)]
Key points - R-Hip : [(764, 928, 0.42972657)]
Key points - R-Knee : [(1013, 1259, 0.14846112), (556, 1218, 0.66932285)]
Key points - R-Ankle : [(557, 1507, 0.7041707)]
Key points - L-Hip : [(1012, 887, 0.47172996)]
Key points - L-Knee : [(1136, 1218, 0.3460872)]
Key points - L-Ankle : [(1301, 1548, 0.5098222)]
Key points - R-Eye : [(929, 309, 0.8521562)]
Key points - L-Eye : [(1012, 309, 0.8620853)]
Key points - R-Ear : [(888, 309, 0.80331016)]
Key points - L-Ear : [(1093, 309, 0.7380304)]

Valid Pairs: [array([[1.        , 2.        , 0.86599229]]), array([[1.        , 5.        , 0.79962465]]), array([[2.  

In [22]:
person_wise_keypoints = assign_keypoints(valid_pairs, invalid_pairs)

for i in range(17):
    for n in range(len(person_wise_keypoints)):
        idx = person_wise_keypoints[n][np.array(POSE_PAIRS[i])]
        
        if -1 in idx:
            continue
        
        B = np.int32(key_points_list[idx.astype(int), 0])
        A = np.int32(key_points_list[idx.astype(int), 1])
        
        cv2.line(img_copy, (B[0], A[0]), (B[1], A[1]), colors[i], 3, cv2.LINE_AA)
        
cv2.imshow('Bodyline Pose', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
proto_file = './openpose/openpose-master/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt'
weight_file = './openpose/openpose-master/models/pose/mpi/pose_iter_160000.caffemodel'

frame_width, frame_height = img.shape[1], img.shape[0]
in_height = 368
in_width = 368

pose_net = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
pose_blob = cv2.dnn.blobFromImage(img_copy, scalefactor=1.0/255, size=(in_width, in_height), mean=(0,0,0), swapRB=False, crop=False)
pose_net.setInput(pose_blob)
output = pose_net.forward()

img = cv2.imread('./data/basketball.jpg')
img_copy = img.copy()

h, w = output.shape[2], output.shape[3]
threshold = 0.1
n_points = 44 # MPI Pose

body_points = []
for i in range(n_points):
    probs = output[0,i,:,:]
    min_val, prob, min_loc, point = cv2.minMaxLoc(probs)
    
    if prob > threshold:
        
        x = (frame_width * point[0]) / w
        y = (frame_height * point[1]) / h
        
        cv2.circle(img_copy, (int(x), int(y)), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(img_copy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, lineType=cv2.LINE_AA)
        
        body_points.append((int(x), int(y)))
        
    else:
        body_points.append(None)
        
cv2.imshow('Body Points', img_copy)    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
body_points